In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tarfile
from six.moves import urllib
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Get the data


In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z


In [ ]:
!unzip -o /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip -o /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

In [ ]:
def printShapeAndData(panda):  
    print(panda.shape)
    display(panda.head(10))
data = pd.read_csv('train.tsv', sep='\t')

printShapeAndData(data)

In [ ]:
#train id is not needed so we remove it 
data.drop(columns =["train_id"], inplace = True)

In [ ]:
printShapeAndData(data)

# Cleaning data

In [ ]:
pd.isnull(data).sum()

In [ ]:
data.brand_name=data.brand_name.fillna(value = "no brand defined")

data.category_name=data.category_name.fillna(value = "Unknown/Unknown/Unknown")

data.item_description=data.item_description.fillna(value = "No description yet")

In [ ]:
pd.isnull(data).sum()

In [ ]:
data.info()

# data analytical

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
data.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
#spliting the categories to make better plots and avoid noise in the training


# new data frame with split value columns
new = data["category_name"].str.split("/", n = 2, expand = True)
print(new)
for index,catergory in enumerate(new):
    data[f"category {index+1}"]=new[index]

# Dropping old Name columns
data.drop(columns =["category_name"], inplace = True)

data

In [ ]:
data["category 1"].value_counts().plot(kind='bar')

In [ ]:
data["category 2"].value_counts().plot(kind='bar')

In [ ]:
data["category 3"].value_counts().plot(kind='bar')

In [ ]:
data["brand_name"].value_counts().plot(kind='bar')

In [ ]:
#data["item_description"].value_counts().plot(kind='bar')

# preparing for machine learning

In [ ]:
cat_col=data.select_dtypes(include=['object']).columns.tolist()
cat_col

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder_X=LabelEncoder()
for col in cat_col:
    data[col] = labelencoder_X.fit_transform(data[col])
data[cat_col]

In [ ]:
corr_matrix = data.corr()


In [ ]:
import seaborn as sns

sns.heatmap(corr_matrix, 
            xticklabels=corr_matrix.columns.values,
            yticklabels=corr_matrix.columns.values)

In [ ]:
if 'price'in data.columns:
    data_targets=data.pop('price')
data_targets

In [ ]:
data

# spilliting data to train and test

In [ ]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target = train_test_split(data,data_targets, test_size=0.2)


In [ ]:
printShapeAndData(train_data)

In [ ]:
printShapeAndData(train_target)

In [ ]:

printShapeAndData(test_data)

In [ ]:
printShapeAndData(test_target)

# machine learning

In [ ]:
from sklearn.linear_model import LinearRegression,SGDRegressor,ElasticNet,Ridge
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error



In [ ]:
def checkModelPerformane(model):
    X=train_data
    y=train_target
    model.fit(X,y)
    pred=model.predict(test_data)
    print("mean_squared_error: ",np.sqrt(mean_squared_error(test_target, pred))) 
    print("mean_absolute_error: ", np.sqrt(mean_absolute_error(test_target, pred))) 


In [ ]:
print("LinearRegression")
checkModelPerformane(LinearRegression())

In [ ]:
print("LinearRegression with lasso regualrizion")
checkModelPerformane(linear_model.Lasso(alpha=0.1))

In [ ]:
print("lasso regression ")
checkModelPerformane(linear_model.Lasso(alpha=0.1))

In [ ]:
print("ElasticNet regression ")
checkModelPerformane(ElasticNet())

In [ ]:
print("Ridge regression ")
checkModelPerformane(Ridge(alpha=1.0))

In [ ]:
print("Random Forest")
checkModelPerformane(RandomForestRegressor())
